In [3]:
# Importing modules

import pandas as pd
import numpy as np

In [2]:
# Treating data

# Loading data
df = pd.read_parquet("BTC_features_clean.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27203 entries, 2017-12-15 22:00:00 to 2024-12-31 23:00:00
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                27203 non-null  float64
 1   high                27203 non-null  float64
 2   low                 27203 non-null  float64
 3   close               27203 non-null  float64
 4   volume              27203 non-null  float64
 5   quote_asset_volume  27203 non-null  float64
 6   number_of_trades    27203 non-null  float64
 7   taker_buy_base      27203 non-null  float64
 8   taker_buy_quote     27203 non-null  float64
 9   sma_7d              27203 non-null  float64
 10  sma_30d             27203 non-null  float64
 11  sma_50d             27203 non-null  float64
 12  sma_100d            27203 non-null  float64
 13  return              27203 non-null  float64
 14  volatility_20       27203 non-null  float64
 15  volatility_50     

In [9]:
# Transforming data into inputs for the model

def make_windows(df, input_len=168, output_horizons=[1, 6, 12, 24, 168], target_col="close"):
    """
    Génère les fenêtres (X, Y) pour l'entraînement d'un modèle séquentiel.
    
    df : DataFrame Pandas (index = datetime, colonnes = features)
    input_len : longueur de la fenêtre d'entrée (en heures)
    output_horizons : horizons de prédiction (en heures)
    target_col : colonne de référence pour la target (ex. 'close')
    
    Retourne :
        X : np.array (n_samples, input_len, n_features)
        Y : np.array (n_samples, len(output_horizons))
    """
    feature_cols = [c for c in df.columns if c != target_col]
    data = df[feature_cols].values
    target = df[target_col].values
    n_features = data.shape[1]

    X, Y = [], []
    max_h = max(output_horizons)

    for t in range(len(df) - input_len - max_h):
        # Fenêtre d'entrée
        x_window = data[t : t + input_len]

        # Targets en rendements relatifs
        y_window = []
        current_price = target[t + input_len - 1]
        for h in output_horizons:
            future_price = target[t + input_len + h - 1]
            y_window.append(future_price / current_price - 1)

        X.append(x_window)
        Y.append(y_window)

    X = np.array(X, dtype=np.float32)
    Y = np.array(Y, dtype=np.float32)

    print(f"✅ make_windows: X={X.shape}, Y={Y.shape}")
    return X, Y

In [10]:
# Inputs and Outputs generations for the whole dataset
X, Y = make_windows(df, input_len=168, output_horizons=[1, 6, 12, 24, 168], target_col="close")

✅ make_windows: X=(26867, 168, 24), Y=(26867, 5)


In [12]:
# Split the dataset for train, validation and test.

def time_series_split(X, Y, train_size=0.7, val_size=0.15):
    """
    Split temporel des données en train, val et test.
    """
    n = len(X)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))
    
    X_train, Y_train = X[:train_end], Y[:train_end]
    X_val,   Y_val   = X[train_end:val_end], Y[train_end:val_end]
    X_test,  Y_test  = X[val_end:], Y[val_end:]
    
    print(f"Train: {X_train.shape}, {Y_train.shape}")
    print(f"Val:   {X_val.shape}, {Y_val.shape}")
    print(f"Test:  {X_test.shape}, {Y_test.shape}")
    
    return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test)

In [13]:
(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = time_series_split(X, Y)

Train: (18806, 168, 24), (18806, 5)
Val:   (4030, 168, 24), (4030, 5)
Test:  (4031, 168, 24), (4031, 5)


## INFORMER CREATION

### Etape 0: Normalization of the features

On utilise ici des moyennes sur le train car c'est la norme d'utilisation pour éviter le data leaking.

In [23]:
import numpy as np

# Suppose que tu as déjà X, Y puis le split
(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = time_series_split(X, Y)

# Stats uniquement sur le train (pas de fuite d'info)
feat_mean = X_train.mean(axis=(0,1), keepdims=True)
feat_std  = X_train.std(axis=(0,1), keepdims=True) + 1e-8

X_train_n = (X_train - feat_mean) / feat_std
X_val_n   = (X_val   - feat_mean) / feat_std
X_test_n  = (X_test  - feat_mean) / feat_std


Train: (18806, 168, 24), (18806, 5)
Val:   (4030, 168, 24), (4030, 5)
Test:  (4031, 168, 24), (4031, 5)


### Etape 1 : Conding ProbSparse attention -- Multi-Head attention for Informers.

On regarde chaque étape du call de la classe

#### Etape 0 :

On récupère dans `B` et `T` la taille du batch et la taille de la séquence respectivement.

#### Etape 1 :

On commence par projetter linéairement `X` sur `Q`, `K` et `V` à l'aide de différentes matrices. Mathématique, on a:
$$
Q = XW_Q, \quad K = XW_K, \quad V = XW_V
$$

X étant de shape `(B, T, d_mondel ou d_in)`, nous obtenons des objets `Q`, `K` et `V` de taille `(B, T, d_model)`.

Les poids des matrices $W_Q$, $W_K$ et $W_V$ (de shape `(d_in, d_model)`) sont entraînables.

#### Etape 2 : 

On divise notre attention en multi-tête, comme pour un transformer. Ce nombre de tête est défini par `num_head`. La valeur par défault de la classe est 4. Ainsi, on passe d'une information de taille `d_model` à `d_k` présent dans chacune des `num_head` têtes, avec `d_k` défini tel que:

$$
d_k = \frac{d_{\text{model}}}{\text{num\_heads}}
$$

La fonction `split_heads` prend en entrée un tenseur et fais cette séparation en `num_head` têtes de taille `d_k`. Ainsi, on l'applique à `Q`, `K` et `V`. Ils deviennent des objets de shape `(B, T, h, d_k)`.

#### Etape 3: 

L'idée est de traîté chaque tête comme une séquence indépendante. Ainsi, on va applatir notre vecteur sur la dimension `B`et `h`, de manière à avoir un objet de shape `(B*h, T, d_k)`. On stocks ces nouveaux objets dans `Q_`, `K_` et `V_` pour respectivement les objets `Q`, `K` et `V`.

#### Etape 4:

D'abord, on applique une régularisation L2 permettant de réduire l'overfitting sur `Q_` et `K_`, que l'on stock respectivement dans `norm_q` et `norm_k`. Il est important que, dans ce cas, nous avons:
$$
||q_i|| = ||k_j|| = 1 \,
$$
avec $||q_i||$ et $||k_j||$, les éléments $i$ et $j$ correspondant à la dimension `T` de `norm_q` et `norm_k` respectivement.

Ensuite, nous pouvons calculer les scores pour les objets `norm_q` et `norm_k`. On nomme ceci `scores` car on regarde, pour chaque $q_i$, un score de similarité avec chaque $k_j$. Les scores sont calculés de cette manière :
$$
s_i = \text{max} \frac{q_i \cdot k_j}{||q_i|| \times ||k_j||}
$$

Dans notre code, nous calculons d'abord tous les $s_{i,j}$, et ensuite nous prenons le $s_i$ max sur pour chaque $j$ avec respectivement les variales `scores`et `max_scores`.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class ProbSparseAttention(layers.Layer):
    """
    Version pédagogique de ProbSparse Self-Attention.
    Sélectionne seulement les top-u queries les plus informatives.
    """
    def __init__(self, d_model, num_heads=4, dropout=0.1, u=10, **kwargs):
        super().__init__(**kwargs)
        assert d_model % num_heads == 0, "d_model doit être divisible par num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        self.dropout = dropout
        self.u = u  # nombre de queries gardées

        # Projections linéaires
        self.W_q = layers.Dense(d_model)
        self.W_k = layers.Dense(d_model)
        self.W_v = layers.Dense(d_model)
        self.W_o = layers.Dense(d_model)

        self.drop = layers.Dropout(dropout)

    def call(self, x, training=False):
        """
        x: (batch, T, d_model)
        """
        # 0. Récupération de la taille du batch B et de la séquence T
        B, T, _ = tf.unstack(tf.shape(x))

        # 1. Projeter en Q, K, V
        Q = self.W_q(x)  # (B, T, d_model)
        K = self.W_k(x)
        V = self.W_v(x)

        # 2. Découper en têtes
        def split_heads(tensor):
            return tf.reshape(tensor, (B, T, self.num_heads, self.d_k))  # (B, T, h, d_k)

        Q = split_heads(Q)
        K = split_heads(K)
        V = split_heads(V)

        # 3. Aplatir batch*têtes
        Q_ = tf.reshape(Q, (B*self.num_heads, T, self.d_k)) # (B*h, T, d_k)
        K_ = tf.reshape(K, (B*self.num_heads, T, self.d_k))
        V_ = tf.reshape(V, (B*self.num_heads, T, self.d_k))

        # 4. Calcul score de sparsité (max similitude)
        norm_q = tf.nn.l2_normalize(Q_, axis=-1)
        norm_k = tf.nn.l2_normalize(K_, axis=-1)
        scores = tf.matmul(norm_q, norm_k, transpose_b=True)  # (B*h, T, T)
        max_scores = tf.reduce_max(scores, axis=-1)  # (B*h, T)

        # 5. Top-u queries
        u = tf.minimum(self.u, T)
        top_idx = tf.argsort(max_scores, axis=-1, direction="DESCENDING")[:, :u]  # (B*h, u)

        # 6. Extraire les queries sélectionnées
        Q_sel = tf.gather(Q_, top_idx, batch_dims=1)  # (B*h, u, d_k)

        # 7. Attention seulement sur Q_sel vs tous K,V
        attn_weights = tf.matmul(Q_sel, K_, transpose_b=True) / tf.math.sqrt(tf.cast(self.d_k, tf.float32))
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
        attn_weights = self.drop(attn_weights, training=training)

        out = tf.matmul(attn_weights, V_)  # (B*h, u, d_k)

        # 8. Reconstruire la séquence complète (remplir les queries non sélectionnées par 0)
        out_full = tf.zeros_like(Q_)  # (B*h, T, d_k)

        # indices pour scatter update
        batch_idx = tf.repeat(tf.range(B*self.num_heads)[:, None], u, axis=1)  # (B*h, u)
        idx = tf.stack([batch_idx, top_idx], axis=-1)  # (B*h, u, 2)

        out_full = tf.tensor_scatter_nd_update(out_full, tf.reshape(idx, (-1, 2)), tf.reshape(out, (-1, self.d_k)))

        # 9. Reconstruire (B, T, d_model)
        out_full = tf.reshape(out_full, (B, T, self.d_model))
        return self.W_o(out_full)


Ici, on fait comme avec le transformer. Se référer à la documentation. On code le **positional encoding**.

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class SinusoidalPE(layers.Layer):
    """Encodage positionnel sin/cos (Vaswani et al., 2017)."""
    def __init__(self, d_model, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model

    def call(self, x):
        # x: (B, T, d_model) — B=batch, T=longueur séquence, d_model=dim embedding
        T = tf.shape(x)[1]
        d = self.d_model

        # positions: [0, 1, ..., T-1]  shape (T, 1)
        pos = tf.cast(tf.range(T)[:, None], tf.float32)

        # indices de dimensions: [0, 1, ..., d-1] shape (1, d)
        i = tf.cast(tf.range(d)[None, :], tf.float32)

        # angle_rates = 1 / 10000^{2i/d}
        angle_rates = tf.pow(10000.0, - (tf.floor(i/2.0) * 2.0) / tf.cast(d, tf.float32))
        # angle_rads = pos * angle_rates  shape (T, d)
        angle_rads = pos * angle_rates

        # Appliquer sin sur dimensions paires (2k), cos sur impaires (2k+1)
        sines = tf.sin(angle_rads[:, 0::2])
        coses = tf.cos(angle_rads[:, 1::2])

        # Recomposer en (T, d): intercaler sin/cos
        pe = tf.concat([tf.reshape(tf.stack([sines, coses], axis=-1), (T, -1)),
                        tf.zeros((T, d - tf.shape(tf.reshape(tf.stack([sines, coses], axis=-1), (T, -1)))[1]))], axis=-1)
        pe = pe[:, :d]  # sécurité si d est impair

        return x + pe[None, ...]  # broadcast sur le batch


2025-09-03 15:14:20.686040: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
